# Packages

In [ ]:
# Matplotlib setup
from matplotlib_setup import mpl_setup

# Modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager

# Setup

In [ ]:
# directories
wd = '/Users/lneumann/Documents/'
pystruct_dir = wd + 'Data/ALMOND/pystruct/'
plot_dir = wd + 'Products/ALMOND/dense_gas_letter/plots/appendix/'

# galaxies
galaxies = ['ngc0628', 'ngc2903', 'ngc4321']

# Plotting: average spectra

In [ ]:
savepng = False
savepdf = True

# maximum radius to be included in the average spectra
rgal_max = 5 

# y-axis data range
ylim = [-1,3.5]

# make figure
mpl_setup(figtype='paper-1/2', aspect=1.5)
fig = plt.figure()
fig.subplots_adjust(hspace=0.15, wspace=0)

# loop over galaxies
for i, galaxy in enumerate(galaxies):
    
    # load pystructure database
    pystruct_file = pystruct_dir + '%s_data_struct_33.3as_spb2_empire_vs_almond.npy' % galaxy
    struct = np.load(pystruct_file, allow_pickle = True).item()
    
    # galactocentric radius
    rgal = struct['rgal_kpc']
    
    # intensities
    int_hcn_almond = struct['INT_VAL_HCN10_ALMOND']
    int_hcn_almond = struct['INT_VAL_HCN10_ALMOND']
    
    # spectra
    spec_hcn_almond = struct['SPEC_VAL_HCN10_ALMOND'] * 1e3  # mK
    spec_hcn_empire = struct['SPEC_VAL_HCN10_EMPIRE'] * 1e3  # mK
    spec_co21_alma = struct['SPEC_VAL_CO21_ALMA'] * 1e3  # mK
    rms_hcn_almond = struct['SPEC_RMS_HCN10_ALMOND'] * 1e3  # mK
    rms_hcn_empire = struct['SPEC_RMS_HCN10_EMPIRE'] * 1e3  # mK
    rms_co21_alma = struct['SPEC_RMS_CO21_ALMA'] * 1e3  # mK
    
    # compute average spectra
    spec_hcn_almond_avg = np.nanmean(spec_hcn_almond[rgal < rgal_max], axis=0)
    spec_hcn_empire_avg = np.nanmean(spec_hcn_empire[rgal < rgal_max], axis=0)
    spec_co21_alma_avg = np.nanmean(spec_co21_alma[rgal < rgal_max], axis=0)
    rms_co21_alma_avg = np.nanmean(rms_co21_alma[rgal < rgal_max])

    # velocity axis
    v_pix0 = struct['SPEC_CRPIX_HCN10_ALMOND']
    v_ch0 = struct['SPEC_VCHAN0_HCN10_ALMOND']
    v_channel = struct['SPEC_DELTAV_HCN10_ALMOND']
    N_ch = len(struct['SPEC_VAL_HCN10_ALMOND'][0])
    vaxis = (v_ch0 + (np.arange(N_ch)+(1-v_pix0)) * v_channel) * 1e-3  # km/s
    vaxis_almond = vaxis[~np.isnan(spec_hcn_almond_avg)]  # ALMOND spectral axis is shorter
    
    
    # create subplot    
    ax = fig.add_subplot(3,1,i+1)

    # plot spectra
    ax.axhline(0, c='k', ls='-')
    ax.step(vaxis, spec_hcn_almond_avg, where='mid', c='tab:blue', lw=1, label='ALMOND\nHCN(1$-$0)', zorder=2)
    ax.step(vaxis, spec_hcn_empire_avg, where='mid', c='tab:red', lw=1, label='EMPIRE\nHCN(1$-$0)', zorder=1)
    ax.step(vaxis, spec_co21_alma_avg/20, where='mid', c='grey', ls='dashed', lw=1, label='ALMA\nCO(2$-$1)/20', zorder=0)

    # noise level
    # ax.axhline(rms_co21_alma_avg/20, c='grey', ls='dotted', zorder=0)
    
    # integration mask
    mask = spec_co21_alma_avg/rms_co21_alma_avg > 2
    vaxis_mask = vaxis[mask == 1]
    vaxis_mask_lim = (vaxis_mask[0]-v_channel*1e-3/2, vaxis_mask[-1]+v_channel*1e-3/2)
    ax.axvspan(vaxis_mask_lim[0], vaxis_mask_lim[1], ec='k', ls='-', fc='slategrey', alpha=.2, lw=0, zorder=0)    
    ax.axvline(vaxis_mask_lim[0], c='k', ls='dashed', lw=1, zorder=20)
    ax.axvline(vaxis_mask_lim[1], c='k', ls='dashed', lw=1, zorder=20)
    # fill spectra over integration range
    # ax.fill_between(vaxis_kms, spec_co, step='mid', y2=0, where=(mask == 1), lw=0, alpha=0.3, color='tab:red')
    # ax.fill_between(vaxis_kms, spec_hcn*30, step='mid', y2=0, where=(mask == 1), lw=0, alpha=0.3, color='tab:blue')

    # compute integrated intensities over average spectra
    int_hcn_almond_avg = np.nansum(spec_hcn_almond_avg * mask) * v_channel * 1e-3 # K.km/s
    int_hcn_empire_avg = np.nansum(spec_hcn_empire_avg * mask) * v_channel * 1e-3 # K.km/s

    
    int_hcn_almond_avg_err = np.nanstd(spec_hcn_almond_avg[mask == 0]) * sum(mask == 1) * v_channel * 1e-3 # K.km/s
    int_hcn_empire_avg_err = np.nanstd(spec_hcn_empire_avg[mask == 0]) * sum(mask == 1) * v_channel * 1e-3 # K.km/s
    
    print(int_hcn_almond_avg, int_hcn_almond_avg_err)
    print(int_hcn_empire_avg, int_hcn_empire_avg_err)

    # axis settings
    xrange = np.nanmax(vaxis_almond) - np.nanmin(vaxis_almond)
    xlim = [np.nanmin(vaxis_almond)-0.1*xrange, np.nanmax(vaxis_almond)+0.1*xrange]
    ymin = np.nanmin([np.nanmin(spec_hcn_almond_avg), np.nanmin(spec_hcn_empire_avg), np.nanmin(spec_co21_alma_avg/20)])
    ymax = np.nanmax([np.nanmax(spec_hcn_almond_avg), np.nanmax(spec_hcn_empire_avg), np.nanmax(spec_co21_alma_avg/20)])
    ylim = [1.5*ymin, 1.1*ymax]
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    if i == 2:
        ax.set_xlabel('Velocity [km\,s$^{-1}$]')
    if i == 1:
        ax.set_ylabel('Brightness temperature [mK]')

    if i == 0:
        mpl.rcParams['text.usetex'] = False
        font = font_manager.FontProperties(family='Arial', style='normal', size='x-small')
        legend1 = ax.legend(loc='upper left', prop=font)
        mpl.rcParams['text.usetex'] = True
    
    if i == 2:
        # data info
        ax.text(0.05, 0.92, 'Average spectra at\nmatched spatial and\nspectral resolution\nacross $r_\mathrm{gal} \leq %.0f\,$kpc' % rgal_max, ha='left', va='top', transform=ax.transAxes, fontsize='xx-small')
    
    # galaxy name
    mpl.rcParams['text.usetex'] = False
    font = font_manager.FontProperties(family='Arial', style='normal', size='medium')
    ax.text(0.97, 0.92, r'NGC$\,$%s' % galaxy[3:], ha='right', va='top', transform=ax.transAxes, fontproperties=font, fontweight='bold')
    mpl.rcParams['text.usetex'] = True

# save plot
savepath = plot_dir + 'ALMOND_vs_EMPIRE_spectra'
if savepng:
    plt.savefig(savepath + '.png', facecolor='white', transparent=False)
if savepdf:
    plt.savefig(savepath + '.pdf', facecolor='white', transparent=False)

plt.show()

# Integrated intensities (pixel-by-pixel)

In [ ]:
savepng = False
savepdf = True

# plotting range
xlim = [1e-2, 5e0]
ylim = [1e-2, 5e0]

# make figure
mpl_setup(figtype='paper-1/2', aspect=1.5)
fig = plt.figure()
fig.subplots_adjust(hspace=0, wspace=0)

# loop over galaxies
for i, galaxy in enumerate(galaxies):
    
    # load pystructure database
    pystruct_file = pystruct_dir + '%s_data_struct_33.3as_spb2_empire_vs_almond.npy' % galaxy
    struct = np.load(pystruct_file, allow_pickle = True).item()
    
    # galactocentric radius
    rgal = struct['rgal_kpc']
    
    # intensities
    int_hcn_almond = struct['INT_VAL_HCN10_ALMOND']
    int_hcn_empire = struct['INT_VAL_HCN10_EMPIRE']
    int_hcn_almond_err = struct['INT_UC_HCN10_ALMOND']
    int_hcn_empire_err = struct['INT_UC_HCN10_EMPIRE']
    
    # spectra
    spec_hcn_almond = struct['SPEC_VAL_HCN10_ALMOND'] * 1e3  # mK
    spec_hcn_empire = struct['SPEC_VAL_HCN10_EMPIRE'] * 1e3  # mK
    spec_co21_alma = struct['SPEC_VAL_CO21_ALMA'] * 1e3  # mK
    rms_hcn_almond = struct['SPEC_RMS_HCN10_ALMOND'] * 1e3  # mK
    rms_hcn_empire = struct['SPEC_RMS_HCN10_EMPIRE'] * 1e3  # mK
    rms_co21_alma = struct['SPEC_RMS_CO21_ALMA'] * 1e3  # mK

    
    # create subplot    
    ax = fig.add_subplot(3,1,i+1)

    # plot spectra
    mask = (int_hcn_almond/int_hcn_almond_err > 3) & (int_hcn_empire/int_hcn_empire_err > 3)
    ax.scatter(int_hcn_almond[mask], int_hcn_empire[mask], s=20, c='k', zorder=1)
    ax.scatter(int_hcn_almond[mask], int_hcn_empire[mask], s=10, c='tab:orange', zorder=2)
    ax.scatter([1e-3], [1e-3], ec='k', fc='tab:orange', lw=0.75, s=1.75*10 , label=r'$\mathrm{S/N}\geq 3$') # for legend
    ax.scatter(int_hcn_almond[~mask], int_hcn_empire[~mask], s=10, c='grey', zorder=0, label=r'$\mathrm{S/N}<3$')

    # 1-to-1 line
    ax.plot(np.logspace(-4, 1, 10), np.logspace(-4, 1, 10), ls='--', c='k', label='$1-1$ line', zorder=5)
    # 50% deviation
    ax.plot(np.logspace(-4, 1, 10), np.logspace(-4, 1, 10)*1.5, ls='dotted', c='k', zorder=5, label=r'50$\%$ deviation')
    ax.plot(np.logspace(-4, 1, 10), np.logspace(-4, 1, 10)/1.5, ls='dotted', c='k', zorder=5)

    # axis settings
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    if i == 2:
        ax.set_xlabel(r'$W_{\mathrm{HCN}(1-0)}\;[\mathrm{K}\,\mathrm{km}\,\mathrm{s}^{-1}]$, ALMOND', loc='right')
    if i == 1:
        ax.set_ylabel(r'$W_{\mathrm{HCN}(1-0)}\;[\mathrm{K}\,\mathrm{km}\,\mathrm{s}^{-1}]$, EMPIRE')

    if i in [0,1]:
        ax.tick_params(labelbottom=False)

    # legend
    if i == 0:
        mpl.rcParams['text.usetex'] = False
        font = font_manager.FontProperties(family='Arial', style='normal', size='x-small')
        legend1 = ax.legend(loc='lower right', prop=font)
        mpl.rcParams['text.usetex'] = True

    # galaxy name
    mpl.rcParams['text.usetex'] = False
    font = font_manager.FontProperties(family='Arial', style='normal', size='medium')
    ax.text(0.03, 0.92, r'NGC$\,$%s' % galaxy[3:], ha='left', va='top', transform=ax.transAxes, fontproperties=font, fontweight='bold')
    mpl.rcParams['text.usetex'] = True

# save plot
savepath = plot_dir + 'ALMOND_vs_EMPIRE_integrated_intensities'
if savepng:
    plt.savefig(savepath + '.png', facecolor='white', transparent=False)
if savepdf:
    plt.savefig(savepath + '.pdf', facecolor='white', transparent=False)

plt.show()

# Brightness temperatures (voxel-by-voxel)

In [ ]:
savepng = False
savepdf = True

# plotting range
xlim = [1e-4, 5e-2]
ylim = [1e-4, 5e-2]

# make figure
mpl_setup(figtype='paper-1/2', aspect=1.5)
fig = plt.figure()
fig.subplots_adjust(hspace=0, wspace=0)

# loop over galaxies
for i, galaxy in enumerate(galaxies):
    
    # load pystructure database
    pystruct_file = pystruct_dir + '%s_data_struct_33.3as_spb2_empire_vs_almond.npy' % galaxy
    struct = np.load(pystruct_file, allow_pickle = True).item()
    
    # galactocentric radius
    rgal = struct['rgal_kpc']
    
    # intensities
    int_hcn_almond = struct['INT_VAL_HCN10_ALMOND']
    int_hcn_empire = struct['INT_VAL_HCN10_EMPIRE']
    int_hcn_almond_err = struct['INT_UC_HCN10_ALMOND']
    int_hcn_empire_err = struct['INT_UC_HCN10_EMPIRE']
    
    # spectra
    spec_hcn_almond = struct['SPEC_VAL_HCN10_ALMOND']  # mK
    spec_hcn_empire = struct['SPEC_VAL_HCN10_EMPIRE']  # mK
    rms_hcn_almond = struct['SPEC_RMS_HCN10_ALMOND']   # mK
    rms_hcn_empire = struct['SPEC_RMS_HCN10_EMPIRE']  # mK

    # broad rms to intensity shape
    rms_hcn_almond_2d = np.broadcast_to(rms_hcn_almond, np.shape(spec_hcn_almond.T)).T
    rms_hcn_empire_2d = np.broadcast_to(rms_hcn_empire, np.shape(spec_hcn_empire.T)).T

    
    # create subplot    
    ax = fig.add_subplot(3,1,i+1)

    # plot spectra
    mask = (spec_hcn_almond/rms_hcn_almond_2d > 3) & (spec_hcn_empire/rms_hcn_empire_2d > 3)
    # mask = mask.T
    # mask = spec_hcn_almond > 0
    ax.scatter(spec_hcn_almond[mask], spec_hcn_empire[mask], s=20, c='k', zorder=1)
    ax.scatter(spec_hcn_almond[mask], spec_hcn_empire[mask], s=10, c='tab:green', zorder=2)
    ax.scatter([1e-3], [1e-3], ec='k', fc='tab:green', lw=0.75, s=1.75*10 , label=r'$\mathrm{S/N}\geq 3$') # for legend
    ax.scatter(spec_hcn_almond[~mask], spec_hcn_empire[~mask], s=10, c='grey', zorder=0, label=r'$\mathrm{S/N}<3$')

    # 1-to-1 line
    ax.plot(np.logspace(-4, 3, 10), np.logspace(-4, 3, 10), ls='--', c='k', label='$1-1$ line', zorder=5)
    # 50% deviation
    ax.plot(np.logspace(-4, 3, 10), np.logspace(-4, 3, 10)*1.5, ls='dotted', c='k', zorder=5, label=r'50$\%$ deviation')
    ax.plot(np.logspace(-4, 3, 10), np.logspace(-4, 3, 10)/1.5, ls='dotted', c='k', zorder=5)

    # axis settings
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    if i == 2:
        ax.set_xlabel(r'$T_{\mathrm{HCN}(1-0)}\;[\mathrm{K}]$, ALMOND', loc='right')
    if i == 1:
        ax.set_ylabel(r'$T_{\mathrm{HCN}(1-0)}\;[\mathrm{K}]$, EMPIRE')

    if i in [0,1]:
        ax.tick_params(labelbottom=False)

    # legend
    if i == 0:
        mpl.rcParams['text.usetex'] = False
        font = font_manager.FontProperties(family='Arial', style='normal', size='x-small')
        legend1 = ax.legend(loc='lower right', prop=font, ncol=2)
        mpl.rcParams['text.usetex'] = True
        
    # galaxy name
    mpl.rcParams['text.usetex'] = False
    font = font_manager.FontProperties(family='Arial', style='normal', size='medium')
    ax.text(0.03, 0.92, r'NGC$\,$%s' % galaxy[3:], ha='left', va='top', transform=ax.transAxes, fontproperties=font, fontweight='bold')
    mpl.rcParams['text.usetex'] = True

# save plot
savepath = plot_dir + 'ALMOND_vs_EMPIRE_brightness_temperatures'
if savepng:
    plt.savefig(savepath + '.png', facecolor='white', transparent=False)
if savepdf:
    plt.savefig(savepath + '.pdf', facecolor='white', transparent=False)

plt.show()